In [ ]:
import numpy as np
import random
import math
from copy import copy

In [ ]:
# Player 1 = 1
# Computer is player 2 = 2
# Board is 3x3 matrix 
# Move is a tuple of x - y
# initialMoves = [(0,0), (0,2), (2,0), (2,1), (2,2)]

In [ ]:
# Class desribes a node datastructure containing all the information to create a monte carlo search tree for the tictactoe game
class Node:
  def __init__(self, board, possibleMoves, playerNumber, action, parent=None):
    self.visits = 0
    self.wins = 0
    self.action = action
    self.playerNumber = playerNumber
    self.possibleMoves = possibleMoves
    self.board = board
    self.parent = parent
    self.children = []
    self.q_value = 0

  # creates children for every possble move from this node
  def createChildren(self, node):
    currentPlayer = playerSwap(self.playerNumber)
    for move in self.possibleMoves:
      newBoard = np.array(self.board, copy=True)
      newBoard[move[0]][ move[1]] = currentPlayer
      newPossibleMoves = self.possibleMoves.copy()
      newPossibleMoves.remove(move)
      newNode = Node(newBoard, newPossibleMoves, currentPlayer, move, node)
      self.children.append(newNode)

  # Creates a new child for a move. If move-node doesnt exist. Returns the node
  def createChild(self, node, move):
    for child in self.children:
      if child.action == move:
        return child
    currentPlayer = playerSwap(self.playerNumber)
    newBoard = np.array(self.board, copy=True)
    newBoard[move[0]][ move[1]] = currentPlayer
    newPossibleMoves = self.possibleMoves.copy()
    newPossibleMoves.remove(move)
    newNode = Node(newBoard, newPossibleMoves, currentPlayer, move, node)
    self.children.append(newNode)
    return newNode

  # if a node has children
  def hasChildren(self):
    return 1 if self.children else 0

  # increments the node visit counter
  def incrementVisit(self):
    self.visits+=1

  # increments the node win counter
  def incrementWins(self):
    self.wins+=1

  def getQvalue(self):
    return self.q_value

  # returns the wins counter of the node
  def getWins(self):
    return self.wins

  # returns the visits counter of the node
  def getVisits(self):
    return self.visits

  # returns the boardStatus of the nodes board
  def getBoardStatus(self):
    return boardStatus(self.board)
 
  # check if nodes board is in terminal condition
  def isTerminal(self):
    return 1 if self.getBoardStatus() > 0 else 0

  # returns a copy of the possible moves from this node
  def getPossibleMoves(self):
    return self.possibleMoves.copy()

  # returns the maximumChild (based on win/visit rate) of the node
  def getMaxChild(self):
    maxChildren = []
    maxQval = -100
    for child in self.children:
      if child.q_value > maxQval:
        maxChildren = [child]
        maxQval = child.q_value
      elif child.q_value == maxQval:
        maxChildren.append(child)
    return random.choice(maxChildren)
    
  # returns the minimumChild (based on win/visit rate) of the node
  def getMinChild(self):
    maxChildren = []
    maxQval = 100
    for child in self.children:
      if child.q_value < maxQval:
        maxChildren = [child]
        maxQval = child.q_value
      elif child.q_value == maxQval:
        maxChildren.append(child)
    return random.choice(maxChildren)

  # returns list of wins and visits of all nodes children
  def getChildInformation(self):
    vals = []
    win_rates = []
    for child in self.children:
      vals.append(child.getQvalue())
      win_rates.append(child.wins / (child.visits + 1))
    return vals, win_rates

In [ ]:
# function returns a swapped player number 1 -> 2 and 2 -> 1
def playerSwap(playerNumber):
  return 3 - (playerNumber)

# helper function to pretty print the board
def xOrO(player):
  if player == 1:
    return "X"
  if player == 2:
    return "O"
  return " "

# prints a visual representation of the board matrix
def printBoard(board):
  print('{} | {} | {}'.format(xOrO(board[0,0]), xOrO(board[0,1]), xOrO(board[0,2])))
  print('----------')
  print('{} | {} | {}'.format(xOrO(board[1,0]), xOrO(board[1,1]), xOrO(board[1,2])))
  print('----------')
  print('{} | {} | {}'.format(xOrO(board[2,0]), xOrO(board[2,1]), xOrO(board[2,2])))

# Wrapper to determine if board is in terminal state. uses boardStatus
def isTerminal(board):
  return 1 if boardStatus(board) > 0 else 0

# returns the status of the passed tictactoe board. 0 for non terminal state, 3 for draw and 1, 2 for player win
def boardStatus(board):
  winningState1 = np.array([2,2,2])
  winningState2 = np.array([1,1,1])
  if (board[:,0].T == winningState2).all() or (board.diagonal() == winningState2).all() or (np.fliplr(board).diagonal() == winningState2).all() or (board[2,:] == winningState2).all():
    return 1
  if (board[:,2] == winningState1).all() or (board[0,:] == winningState1).all() or (board[2,:] == winningState1).all():
    return 2
  if np.count_nonzero(board) == 9:
    return 3
  return 0

#  given the passed board, possible moves and playerNumber uses a monte-carlo tree search to choose a move from possibleMoves
def findNextMove(board, playerNumber, possibleMoves):
  opponent = playerSwap(playerNumber)
  root = Node(board, possibleMoves, opponent, 0)
  root.createChildren(root)
  rootLen = len(root.children)
  converged = False
  lastQvalues = [10] * rootLen
  totalIterations = 0

  while not converged:
    node = random.choice(root.children)
    while not node.isTerminal():
      node.incrementVisit()
      if node.isTerminal():
        node =  node
      else:
        move = random.choice(node.getPossibleMoves())
        node = node.createChild(node, move)

    result = node.getBoardStatus()

    # Back-propagation
    while node not in root.children:
      if result == 1:
          node.incrementWins()
      if node.isTerminal():
        if result == 1:
          node.q_value = 1
          node.incrementWins()
        elif result == 2:
          node.q_value = -1
        else:
          node.q_value = 0
      else:
        qTemp = 0
        for child in node.children:
          qTemp += child.q_value
          
        node.q_value = qTemp / len(node.children)
      node = node.parent

    q_value_3 = 0
    if result == 1:
      node.incrementWins()
    # --- This calculates the q_value for one of the possible moves from the root state
    if not node.isTerminal():
      for child in node.children:
        if child.isTerminal():
          q_value_3 += child.q_value
        else:
          child_q = []
          for grandchild in child.children:
            child_q.append(grandchild.q_value)

          if playerNumber == 1:
            q_value_3 += max(child_q)
          elif playerNumber == 2:
            q_value_3 += min(child_q)
      node.q_value = q_value_3 / len(node.children) if len(node.children) > 0 else 0
    else:
      if result == 1:
        node.q_value = 1
      elif result == 2:
        node.q_value = -1
      else:
        node.q_value = 0

    counter = 0
    if totalIterations > 1000:
      for i, child in enumerate(root.children):
        if math.isclose(child.getQvalue(), lastQvalues[i], rel_tol=1e-5):
          counter +=1
        lastQvalues[i] = child.getQvalue()
      if counter == rootLen:
        converged = True
    totalIterations += 1

  qvals, win_rates = root.getChildInformation()
  print("Q-values for the possible actions: ",qvals)
  print("The win probabilities for these possible actions: ", win_rates)
  if playerNumber == 1:
    winner = root.getMaxChild()
  elif playerNumber == 2:
    winner = root.getMinChild()
  return winner.action

In [ ]:
# runs the game, if graphics is on it also prints the board
def runGame(graphicsOn=False, Minmax=False):
   board = np.zeros((3,3))
   board[0,1] = 2
   board[1,0] = 1
   board[1,1] = 1
   board[1,2] = 2
   playerNumber = 2 # Last piece played by opponent
   possibleMoves = [(0,0), (0,2), (2,0), (2,1), (2,2)]

   for p in range(len(possibleMoves)):
     move = 0
     playerNumber = playerSwap(playerNumber)
     if playerNumber == 1:
       move = findNextMove(board, playerNumber, possibleMoves)
     else:
      if Minmax:
        move = findNextMove(board, playerNumber, possibleMoves)
      else:
         move = random.choice(possibleMoves)


     board[move[0]][move[1]] = playerNumber
     possibleMoves.remove(move)
 
     if(graphicsOn):
      print("iteration {} gives the current board: ".format(p + 1))
      printBoard(board)

     gameStatus = boardStatus(board)
     if isTerminal(board):
       if graphicsOn:
        if gameStatus == 1 or gameStatus == 2:
          print("Game won by player ", gameStatus)
        else:
          print("It was a draw")
       return 
   return 

runGame(False, True)




Q-values for the possible actions:  [1.0, 0.875, 1.0, 0.375, 0.875]
The win probabilities for these possible actions:  [0.7525773195876289, 0.6666666666666666, 0.7202072538860104, 0.3036649214659686, 0.6635071090047393]
Q-values for the possible actions:  [-0.3333333333333333, -0.3333333333333333, 1.0, 0.3333333333333333]
The win probabilities for these possible actions:  [0.6614173228346457, 0.46638655462184875, 0.9962546816479401, 0.8629032258064516]
Q-values for the possible actions:  [1, 0.0, 0.0]
The win probabilities for these possible actions:  [316.0, 0.4730878186968839, 0.49404761904761907]


Code inspiration: https://towardsdatascience.com/reinforcement-learning-implement-tictactoe-189582bea542